In [42]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

dataDir = "/code/archive/garbage_classification/Garbage_classification"

imgHeight = 224
imgWidth = 224
batchSize = 32

trainDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

valDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2

IMAGE_SHAPE = (imgHeight, imgWidth)

# Apply normalization to images
normalization_layer = tf.keras.layers.Rescaling(1./255)
trainDs = trainDs.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
valDs = valDs.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

AUTOTUNE = tf.data.AUTOTUNE
trainDs = trainDs.cache().prefetch(buffer_size=AUTOTUNE)
valDs = valDs.cache().prefetch(buffer_size=AUTOTUNE)

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(imgHeight,imgWidth),
])

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(imgHeight, imgWidth, 3),
    trainable=False)

feature_extractor_layer.trainable = False


model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,
  feature_extractor_layer,
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])
# model.summary()




Found 2390 files belonging to 5 classes.
Using 1912 files for training.
Found 2390 files belonging to 5 classes.
Using 478 files for validation.


AttributeError: 'Sequential' object has no attribute 'feature_extractor_layer'

In [29]:
# Fit model

history = model.fit(trainDs,
epochs=10,
validation_data=valDs)

# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

Epoch 1/10
60/60 [==============================] - 22s 358ms/step - loss: 0.8961 - acc: 0.7103 - val_loss: 0.4690 - val_acc: 0.8285
Epoch 2/10
60/60 [==============================] - 21s 355ms/step - loss: 0.3197 - acc: 0.8813 - val_loss: 0.4322 - val_acc: 0.8473
Epoch 3/10
60/60 [==============================] - 22s 362ms/step - loss: 0.2081 - acc: 0.9336 - val_loss: 0.3999 - val_acc: 0.8536
Epoch 4/10
60/60 [==============================] - 21s 348ms/step - loss: 0.1376 - acc: 0.9587 - val_loss: 0.3883 - val_acc: 0.8598
Epoch 5/10
60/60 [==============================] - 22s 362ms/step - loss: 0.0941 - acc: 0.9780 - val_loss: 0.4111 - val_acc: 0.8494
Epoch 6/10
60/60 [==============================] - 21s 357ms/step - loss: 0.0692 - acc: 0.9854 - val_loss: 0.4050 - val_acc: 0.8556
Epoch 7/10
60/60 [==============================] - 21s 357ms/step - loss: 0.0507 - acc: 0.9948 - val_loss: 0.4373 - val_acc: 0.8640
Epoch 8/10
60/60 [==============================] - 21s 351ms/step - 

In [41]:
from PIL import Image
model.save('./saved_mode/garbage_class.h5')
# img = Image.open("/code/test_images/plastic_bottle_1.jpg").convert('L').resize((imgWidth,imgHeight), Image.ANTIALIAS)
# img = np.array(img)
# print(img.shape())

img = tf.io.read_file("/code/test_images/paper_1.jpg")
tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
tensor = tf.image.resize(tensor, [imgHeight, imgWidth])
input_tensor = tf.expand_dims(tensor, axis=0)

prediction = model.predict(input_tensor)
print(prediction)
maxValue = max(prediction[0])
print("Max probability",maxValue)
print("Predicted class",np.where(prediction[0] == maxValue))
print(sum(prediction[0]))

1/1 [==============================] - 0s 41ms/step
[[3.4877878e-05 2.4248541e-06 2.2386787e-06 9.9382579e-01 6.1346185e-03]]
Max probability 0.9938258
Predicted class (array([3]),)
0.9999999532160473
